# Getting started with HEASoftPy
This tutorial provides a quick-start guide to using `heasoftpy`, a Python wrapper for
the high-energy astrophysics software HEASoft.

## Learning Goals
By the end of this tutorial, you will:

- Understand the basic usage of HEASoftPy and the different ways of calling HEASoft tasks.
- Learn about additional options for running pipelines and parallel jobs.

## Introduction
`heasoftpy` is a Python wrapper around the legacy high-energy software suite
HEASoft, which supports analysis for many active and past NASA X-ray and Gamma-ray
missions; it allows HEASoft tools to be called from Python scripts, interactive iPython
sessions, or Jupyter Notebooks.

This tutorial presents a walk through the main features of `heasoftpy`.

### Inputs

- The ObsID of the NuSTAR data used in example 4.
- The ObsIDs of the NICER data used in example 5.

### Outputs

- A filtered pre-processed NICER event list.
- Partially processed NuSTAR data.
- Processed NICER data.

### Runtime
As of 3rd November 2025, this notebook takes ~15 m to run to completion on Fornax, using the 'small' server with 8GB RAM/ 2 cores.

## Imports

In [1]:
import multiprocessing as mp
import os

import heasoftpy as hsp
from astroquery.heasarc import Heasarc

%matplotlib inline

## Global Setup

### Functions

In [2]:
def worker(in_dir: str) -> hsp.core.HSPResult:
    """
    A very simple demonstration of how you can wrap a HEASoftPy task call in order to
    be able to run it on several observations in parallel. In this case the function
    wraps the NICER level-2 processing pipeline, which will prepare data for
    scientific use.

    :param str in_dir: The directory containing the input data.
    :return: The output of the HEASoftPy task.
    :rtype: hsp.core.HSPResult
    """

    with hsp.utils.local_pfiles_context():

        # Call the tasks of interest
        out = hsp.nicerl2(
            indir=in_dir, noprompt=True, clobber=True, geomag_path=GEOMAG_PATH
        )

        # Run any other tasks...

    return out

### Constants

In [3]:
# The ObsID of the NuSTAR data we use in example four
NU_OBS_ID = "60001110002"

# The ObsIDs of the NICER data we use in example five
NI_OBS_IDS = [
    "1010010121",
    "1010010122",
    "1012020112",
    "1012020114",
]

### Configuration

Here we include code that downloads the data for our examples - we don't include it
in the main body of the notebooks as we do not wish it to be the main focus.

In [4]:
# ------------- Configure global package settings --------------
# Raise Python exceptions if a heasoftpy task fails
# TODO Remove once this becomes a default in heasoftpy
hsp.Config.allow_failure = False

# Set up the method for spawning processes.
mp.set_start_method("fork", force=True)
# --------------------------------------------------------------

# ------------- Setting how many cores we can use --------------
NUM_CORES = None
total_cores = os.cpu_count()

if NUM_CORES is None:
    NUM_CORES = total_cores
elif not isinstance(NUM_CORES, int):
    raise TypeError(
        "If manually overriding 'NUM_CORES', you must set it to an integer value."
    )
elif isinstance(NUM_CORES, int) and NUM_CORES > total_cores:
    raise ValueError(
        f"If manually overriding 'NUM_CORES', the value must be less than or "
        f"equal to the total available cores ({total_cores})."
    )
# --------------------------------------------------------------

# Here we make sure we have all the data this notebook requires
if os.path.exists("../../../_data"):
    ROOT_DATA_DIR = os.path.abspath("../../../_data")
    nu_data_dir = os.path.join(ROOT_DATA_DIR, "NuSTAR", "")
    ni_data_dir = os.path.join(ROOT_DATA_DIR, "NICER", "")
else:
    ROOT_DATA_DIR = os.getcwd()
    nu_data_dir = "NuSTAR/"
    ni_data_dir = "NICER/"

nu_data_link = Heasarc.locate_data(
    Heasarc.query_tap(f"SELECT * from numaster where obsid='{NU_OBS_ID}'").to_table(),
    "numaster",
)

# We only download the data if a matching ObsID directory does not exist.
#  This is not a perfect way to determine whether the necessary data are fully
#  present, but it is good enough for this tutorial.
if not os.path.exists(nu_data_dir + f"{NU_OBS_ID}/"):
    # Heasarc.download_data(nu_data_link, location=nu_data_dir)
    Heasarc.download_data(nu_data_link, host="aws", location=nu_data_dir)
    # Heasarc.download_data(nu_data_link, host='sciserver', location=nu_data_dir)

# Construct a string list of NICER ObsIDs to pass to the HEASARC TAP service.
ni_oi_str = "('" + "','".join(NI_OBS_IDS) + "')"
ni_data_links = Heasarc.locate_data(
    Heasarc.query_tap(
        f"SELECT * from nicermastr where obsid IN {ni_oi_str}"
    ).to_table(),
    "nicermastr",
)

# Again, we only download the data if a matching ObsID directory does not exist.
if any([not os.path.exists(os.path.join(ni_data_dir, oi)) for oi in NI_OBS_IDS]):
    # Heasarc.download_data(ni_data_links, location=ni_data_dir)
    Heasarc.download_data(ni_data_links, host="aws", location=ni_data_dir)
    # Heasarc.download_data(ni_data_links, host='sciserver', location=ni_data_dir)

# -------- Get geomagnetic data ---------
# This ensures that geomagnetic data required for NICER analyses are downloaded
GEOMAG_PATH = os.path.join(ROOT_DATA_DIR, "geomag")
os.makedirs(GEOMAG_PATH, exist_ok=True)
out = hsp.nigeodown(outdir=GEOMAG_PATH, allow_failure=False)
# ---------------------------------------

INFO: Downloading data AWS S3 ... [astroquery.heasarc.core]
INFO: Enabling anonymous cloud data access ... [astroquery.heasarc.core]
INFO: downloading s3://nasa-heasarc/nustar/data/obs/00/6/60001110002/ [astroquery.heasarc.core]
INFO: Downloading data AWS S3 ... [astroquery.heasarc.core]
INFO: downloading s3://nasa-heasarc/nicer/data/obs/2017_10/1010010121/ [astroquery.heasarc.core]
INFO: downloading s3://nasa-heasarc/nicer/data/obs/2017_10/1010010122/ [astroquery.heasarc.core]
INFO: downloading s3://nasa-heasarc/nicer/data/obs/2017_10/1012020114/ [astroquery.heasarc.core]
INFO: downloading s3://nasa-heasarc/nicer/data/obs/2017_10/1012020112/ [astroquery.heasarc.core]


INFO: downloading s3://nasa-heasarc/nustar/data/obs/00/6/60001110002/ [astroquery.heasarc.core]


INFO: Downloading data AWS S3 ... [astroquery.heasarc.core]
INFO: downloading s3://nasa-heasarc/nicer/data/obs/2017_10/1010010121/ [astroquery.heasarc.core]


INFO: downloading s3://nasa-heasarc/nicer/data/obs/2017_10/1010010122/ [astroquery.heasarc.core]


INFO: downloading s3://nasa-heasarc/nicer/data/obs/2017_10/1012020114/ [astroquery.heasarc.core]


INFO: downloading s3://nasa-heasarc/nicer/data/obs/2017_10/1012020112/ [astroquery.heasarc.core]


***


## Example 1: Accessing HEASoftPy help files

For general help, you can run `hsp?` (if working in a Jupyter or iPython
notebook) or use `hsp.help()`.

In [5]:
hsp.help()



DESCRIPTION:
-----------
HEASoftpy is a Python package to wrap the HEASoft tools so that
they can be called from python scripts, interactive ipython
sessions, or Jupyter Notebooks.

>>> import heasoftpy as hsp
>>> help(hsp.fdump)
...

>>> result = hsp.ftlist(infile='input.fits', option='T')
>>> print(result.stdout)
...

REQUIREMENTS:
--------------
python (versions later than 3.8)
astropy


EXAMPLES:
--------------
Tasks in heasoftpy can be used in different ways.

- Built-in tasks can be called directly:
>>> result = hsp.ftlist(infile='input.fits', option='T')

** For version 1.4 and above **
To avoid importing all tasks at once (more than 800), the tasks
have been grouped into separate modules.
Wrappers are still available in the heasoftpy.* namespace,
which import from the modules when the task is called.
So you can do lazy (delayed) import with
>>> import heasoftpy as hsp
>>> hsp.ftlist
or full import with
>>> from heasoftpy.heatools import ftlist

To find the corresponding modul

For task-specific help, you can do:

In [6]:
hsp.ftlist?

Alternatively, you may use the HEASoft standard `fhelp`:

In [7]:
hsp.fhelp(task="ftlist")

---------------------
:: Execution Result ::
---------------------
> Return Code: 0
> Output:
NAME

   ftlist - List the contents of the input file.

USAGE

   ftlist infile[ext][filters] option

DESCRIPTION

   'ftlist' displays the contents of the input file. Depending on the
   value of the 'option' parameter, this task can be used to list any
   combination of the following:

     * a one line summary of the contents of each HDU (option = H)
     * the names, units, and ranges of each table column (option = C)
     * all or a selected subset of the header keywords (option = K)
     * the pixel values in a subset of an image (option = I)
     * the values in selected rows and columns of a table (option = T)

   If a specific HDU name or number is given as part of the input file
   name then only that HDU will be listed. If only the root name of the
   file is specified, then every HDU in the file will be listed.

PARAMETERS

   infile [filename]
          Input file name and optiona

```{warning}
Note that the use of '?' is not valid in 'standard' Python, only in Jupyter
notebooks and iPython.
```

## Example 2: Exploring the content of a FITS file with `ftlist`

The simplest way to run a task is call the function directly: `hsp.task_name(...)`.

In this case, it is `hsp.ftlist(...)`

For `ftlist`, there two required inputs: `infile` and `option`, so that both
parameters need to be provided, otherwise, we will get prompted for the missing parameters.

`infile` is the name of the input FITS file. It can be a local or a remote file. In
this case, we use a FITS file from the HEASARC archive. We can specify the header data unit (HDU)
that we want to access from the FITS file in the usual way (e.g., append `[1]` to the file name).

We can also pass other optional parameters (`rows='1-5'` to specify which rows to print).

In [8]:
infile = (
    "https://heasarc.gsfc.nasa.gov/FTP/nicer/data/obs/2017_10/1012010115/"
    "xti/event_cl/ni1012010115_0mpu7_cl.evt.gz[1]"
)
result = hsp.ftlist(infile=infile, option="T", rows="1-5")

The return of all task execution calls is an `HSPResult` object. Which is a convenient
object that holds the status of the call and its return. For example:

- `returncode`: a return code: 0 if the task executed without errors (int).
- `stdout`: standard output (str).
- `stderr`: standard error message (str).
- `params`: dict of the parameters used for the task.
- `custom`: dict of any other variables returned by the task.

In this case, we may want to just print the output as:

In [9]:
print("return code:", result.returncode)
print(result.stdout)

return code: 0

                    TIME RAWX RAWY    PHA PHA_FA DE    DEADTIME EVENT_FL                 TICK MPU_A_ MPU_UNDER_C PI_FAS     PI       PI_RATIO
                       s pixe pixe   chan   chan              s                               Celsiu               chan   chan               
  1  120747738.394123629    4    6   2315   2044 30 0.000022637 00011000       20833975213560 35.700           2    669    754        1.12706
  2  120747739.754077032    5    1   3079   2878 25 0.000022637 00011000      228921045540112 35.169           4   1064   1071        1.00658
  3  120747739.790229082    3    0    824   NULL 27 0.000015195 00010000      228921046472976 35.169           3     NU    181             NU
  4  120747740.091737226    6    6    274   NULL 10 0.000015195 00010000       20840768473294 34.745           3     NU     20             NU
  5  120747740.195529014    1    5    379   NULL 62 0.000015815 00110000       20827191246871 32.519           3     NU     22      

With this, it may be useful to check that `returncode == 0` after every call if you
are not running the tasks interactively.

With `heasoftpy` version 1.5 and above. You can make the call raise a Python exception
when it fails. This feature is controlled by the config parameter: `allow_failure`.

Setting `hsp.config.Config.allow_failure = False`, means the task will raise
an `HSPTaskException` exception if it fails. Setting the value to `True`, means the
task will not raise an exception, and the return code value will need to be checked by the user.

The value is set to `True` by default for versions `<1.5`. For version `1.5`, not
setting the value prints a warning. In a future version, the default will change
to `False`, so that all failures raise an exception.

We can modify the parameters returned in `result`, and pass them again to the task.

Say we do not want to print the column header:

In [10]:
params = result.params
params["colheader"] = "no"
result_no_col_hdr = hsp.ftlist(params)

print(result_no_col_hdr.stdout)

  1  120747738.394123629    4    6   2315   2044 30 0.000022637 00011000       20833975213560 35.700           2    669    754        1.12706
  2  120747739.754077032    5    1   3079   2878 25 0.000022637 00011000      228921045540112 35.169           4   1064   1071        1.00658
  3  120747739.790229082    3    0    824   NULL 27 0.000015195 00010000      228921046472976 35.169           3     NU    181             NU
  4  120747740.091737226    6    6    274   NULL 10 0.000015195 00010000       20840768473294 34.745           3     NU     20             NU
  5  120747740.195529014    1    5    379   NULL 62 0.000015815 00110000       20827191246871 32.519           3     NU     22             NU


If we forget to pass a required parameter, we will be prompted for it. For example:

In [11]:
# result = hsp.ftlist(infile="../tests/test.fits")

would prompt for the `option` value:

```
Print options: H C K I T  [T] ..
```

In this case, parameter `ftlist:option` was missing, so we are prompted for it, and
the default value is printed between brackets: `[T]`, we can type a value and then
press 'Return' to set our own value, or just press 'Return' to accept the default value.

For tasks that take longer to run, the user may be interested in seeing the output as
the task runs. There is a `verbose` option to print the output of the command similar
to the standard output in command line tasks.

In [12]:
result = hsp.ftlist(infile=infile, option="T", rows="1-5", verbose=True)


                    TIME RAWX RAWY    PHA PHA_FA DE    DEADTIME EVENT_FL                 TICK MPU_A_ MPU_UNDER_C PI_FAS     PI       PI_RATIO
                       s pixe pixe   chan   chan              s                               Celsiu               chan   chan               
  1  120747738.394123629    4    6   2315   2044 30 0.000022637 00011000       20833975213560 35.700           2    669    754        1.12706
  2  120747739.754077032    5    1   3079   2878 25 0.000022637 00011000      228921045540112 35.169           4   1064   1071        1.00658
  3  120747739.790229082    3    0    824   NULL 27 0.000015195 00010000      228921046472976 35.169           3     NU    181             NU
  4  120747740.091737226    6    6    274   NULL 10 0.000015195 00010000       20840768473294 34.745           3     NU     20             NU
  5  120747740.195529014    1    5    379   NULL 62 0.000015815 00110000       20827191246871 32.519           3     NU     22             NU


## Example 3: Using `ftselect`

In this second example, we will work with the same `infile` from above.

We see is the first HDU of the file is an events table. Say, we want to filter the
events that have PHA values between 500 and 600.

We can call `hsp.ftselect` like before, but we can also to the call differently by
using `hsp.HSPTask`, and adding the parameters one at a time

In [13]:
# Create a task object
ftselect = hsp.HSPTask("ftselect")

# Pass the input and output files.
ftselect.infile = infile
ftselect.outfile = "tmp.fits"

# Set the selection expression: PHA between 500-600
ftselect.expression = "PHA>500 && PHA<=600"

# We do not want to copy all the file extensions, just the one that is of interest.
ftselect.copyall = False

# We set clobber so the output file is overwritten if it exits.
ftselect.clobber = True

Up to this point, the task has not run yet. We now call `ftselect()` to execute it.

In [14]:
result = ftselect()

Now we can check the content of the new file with `ftlist`:

In [15]:
result = hsp.ftlist(infile="tmp.fits", option="T")
print(result.stdout)


                    TIME RAWX RAWY    PHA PHA_FA DE    DEADTIME EVENT_FL                 TICK MPU_A_ MPU_UNDER_C PI_FAS     PI       PI_RATIO
                       s pixe pixe   chan   chan              s                               Celsiu               chan   chan               
  1  120747745.358898744    6    1    524   NULL 15 0.000015195 00010000       20840904385748 34.851           2     NU     74             NU
  2  120747756.367098957    2    1    550   NULL 36 0.000015195 00010000       20834438978862 35.594           2     NU     92             NU
  3  120747764.666539520    0    1    508    500 56 0.000022637 00011000       20830486740240 32.731           4     80     78       0.975000
  4  120747770.521417022    6    0    548    549 07 0.000022637 00011000       20841835000362 34.109           3     75     74       0.986667
  5  120747776.059803203    0    4    565   NULL 65 0.000015195 00010000       20828116669240 32.413           3     NU     73             NU
  6  

This filtered file contains only PHA values between 500–600!

We'll also clean up after ourselves by deleting the temporary file:

In [16]:
# Now we remove the temporary file
if os.path.exists("tmp.fits"):
    os.remove("tmp.fits")

## Example 4: Parameter query control

For some tasks, particularly pipelines (e.g. `ahpipeline`, `nupipeline`, etc.), the
user may wish to run the task without querying all the parameters. They all have
reasonable defaults.

In that case, we can pass the `noprompt=True` when calling the task, and HEASoftPy
will run the task without checking the parameters. For example, to run the first
stage of processing for the NuSTAR observation `60001110002` (data are downloaded in
the 'configuration' cell near the top of the notebook), we can do:

In [17]:
out = hsp.nupipeline(
    indir=nu_data_dir + f"{NU_OBS_ID}/",
    outdir=f"{NU_OBS_ID}_p",
    steminputs=f"nu{NU_OBS_ID}",
    exitstage=1,
    verbose=True,
    noprompt=True,
    clobber=True,
)


                 Running NuSTAR pipeline
 Task: nupipeline Version: 0.4.12 Release Date: 2023-11-16
nupipeline_0.4.12: Info: CallQuzcif: Running quzcif NuSTAR FPMA - - TELDEF 2012-11-04 17:21:07 "-" retrieve+ clobber=yes
CallQuzcif: Info: Output 'quzcif' Command: 
CallQuzcif: Info:nuA20100101v002.teldef 0
nupipeline_0.4.12: Info: CallQuzcif: Running quzcif NuSTAR FPMB - - TELDEF 2012-11-04 17:21:07 "-" retrieve+ clobber=yes
CallQuzcif: Info: Output 'quzcif' Command: 
CallQuzcif: Info:nuB20100101v002.teldef 0
nupipeline_0.4.12: Info: CallQuzcif: Running quzcif NuSTAR FPM - - ALIGNMENT 2012-11-04 17:21:07 "type.eq.systems" retrieve+ clobber=yes
CallQuzcif: Info: Output 'quzcif' Command: 
CallQuzcif: Info:nuCalign20120101v001.fits 1
------------------------------------------------------------
               Setting Optical Axis Direction
------------------------------------------------------------
nupipeline_0.4.12: Info: 'pntra' and/or 'pntdec' input parameter set to POINT
nupipeline_0.

nupipeline_0.4.12: Info: CallQuzcif: Running quzcif NuSTAR FPMA - - TELDEF 2012-11-04 17:21:07 "-" retrieve+ clobber=yes


CallQuzcif: Info: Output 'quzcif' Command: 
CallQuzcif: Info:nuA20100101v002.teldef 0
nupipeline_0.4.12: Info: CallQuzcif: Running quzcif NuSTAR FPMB - - TELDEF 2012-11-04 17:21:07 "-" retrieve+ clobber=yes


CallQuzcif: Info: Output 'quzcif' Command: 
CallQuzcif: Info:nuB20100101v002.teldef 0
nupipeline_0.4.12: Info: CallQuzcif: Running quzcif NuSTAR FPM - - ALIGNMENT 2012-11-04 17:21:07 "type.eq.systems" retrieve+ clobber=yes


CallQuzcif: Info: Output 'quzcif' Command: 
CallQuzcif: Info:nuCalign20120101v001.fits 1
------------------------------------------------------------
               Setting Optical Axis Direction
------------------------------------------------------------
nupipeline_0.4.12: Info: 'pntra' and/or 'pntdec' input parameter set to POINT
nupipeline_0.4.12: Info: getting default S/C direction
nupipeline_0.4.12: Info: Optical Axis   RA : 3.2188650000000E+02(deg)
nupipeline_0.4.12: Info: Optical Axis  DEC : 5.6889300000000E+01(deg)
nupipeline_0.4.12: Info: Running 'numetrology' 
nupipeline_0.4.12: Command: numetrology clobber=yes outpsdfilecor=./60001110002_p/nu60001110002_psdcorr.fits chatter=3 history=yes mastaspectfile=./60001110002_p/nu60001110002_mast.fits metgridfile=CALDB metflag=yes alignfile=nuCalign20120101v001.fits metrologyfile=/home/jovyan/project/_data/NuSTAR/60001110002/hk/nu60001110002_met.fits.gz inpsdfilecor= psdcal=yes outpsdfile=./60001110002_p/nu60001110002_psd.fits


...

ne
63% done
63% done
63% done
63% done
63% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
64% done
65% done
65% done
65% done
65% done
65% done
65% done
65% done
65% done
65% done
65% done
65% done
65% done
65% done
65% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
66% done
67% done
67% done
67% done
67% done
67% done
67% done
67% done
67% done
67% done
67% done
67% done
67% done
67% done
67% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
68% done
69% done
69% done
69% done
69% done
69% done
69% done
69% done
69% done
69% done
69% done
69% done
69% done
69% done
69% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
70% done
71% done
71% done
71% done
71% don

e
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
95% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
96% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
97% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
98% done
99% done
99% done
99% done
99% done
99% done
99% done
99% done
99% done
99% done
99% done
99% done
99% done
99% done
99% done
100% done


nucoord_0.1.8: Info: 'coordinator' exit with success. X and Y coordinates computed.
nucoord_0.1.8: Info: Moving temporary file '4343_tmp_nucoord/nu60001110002B_uf.1978tmp' to output file './60001110002_p/nu60001110002B_uf.1978tmp'
nucoord_0.1.8: Info: Output Event File './60001110002_p/nu60001110002B_uf.1978tmp' successfully written.
nucoord_0.1.8: Info: Creating Optical Axis File
nucoord_0.1.8: Info: Running 'nuskypos' 
nucoord_0.1.8: Command: nuskypos pntdec=5.6889300000000E+01 initseed=no clobber=yes attfile=./60001110002_p/nu60001110002_att.fits det1yref=350 det1reffile=./60001110002_p/nu60001110002B_det1.fits history=yes chatter=3 instrument=FPMB det1xref=350 optaxisfile=./60001110002_p/nu60001110002B_oa.fits alignfile=nuCalign20120101v001.fits pntra=3.2188650000000E+02 mastaspectfile=./60001110002_p/nu60001110002_mast.fits aberration=no teldef=nuB20100101v002.teldef


---------------------------------------------------------------------
 		Running 'nuskypos_0.1.3'
---------------------------------------------------------------------
nuskypos_0.1.3: Info: Processing nuCalign20120101v001.fits file.
nuskypos_0.1.3: Info: Processing ./60001110002_p/nu60001110002_att.fits file.
nuskypos_0.1.3: Info: './60001110002_p/nu60001110002B_oa.fits' file successfully written.
nuskypos_0.1.3: Info: './60001110002_p/nu60001110002B_det1.fits' file successfully written.
nucoord_0.1.8: Info: 'nuskypos' exit with success. './60001110002_p/nu60001110002B_oa.fits' Optical Axis File created.
----------------------------------------------------------
nucoord_0.1.8: Exit with success 
------------------------------------------------------------
nupipeline_0.4.12: Info: 'nucoord' exit with success. './60001110002_p/nu60001110002B_uf.evt' updated



nupipeline_0.4.12: Stage  I: './60001110002_p/nu60001110002B_uf.evt' calibrated.


                  NuSTAR pipeline Report
 Task: nupipeline Version: 0.4.12 Release Date: 2023-11-16
evt: .... Optical Axis   RA from  RA_PNT keyword : 3.2188650000000E+02(deg)
evt: .... Optical Axis  DEC from DEC_PNT keyword : 5.6889300000000E+01(deg)
FPMA: .... Stage    I: Calibrating Level 1 Event File: './60001110002_p/nu60001110002A_uf.evt'
FPMA: .... Stage    I: Calibrated Event File: './60001110002_p/nu60001110002A_uf.evt'
FPMB: .... Stage    I: Calibrating Level 1 Event File: './60001110002_p/nu60001110002B_uf.evt'
FPMB: .... Stage    I: Calibrated Event File: './60001110002_p/nu60001110002B_uf.evt'

nupipeline_0.4.12: Exit with no errors - Tue Feb  3 11:21:29 EST 2026


    If the NuSTARDAS software was helpful for your research work, the following
    acknowledgement would be appreciated: "This research has made use of the
    NuSTAR Data Analysis Software (NuSTARDAS) jointly developed by the A

## Example 5: Running tasks in parallel

Running HEASoftPy tasks in parallel is straightforward using Python libraries such
as [multiprocessing](https://docs.python.org/3/library/multiprocessing.html). The only subtlety is in the use of parameter files. Many
HEASoft tasks use [parameter file](https://heasarc.gsfc.nasa.gov/ftools/others/pfiles.html) to handle the input parameters.

By defaults, parameters are stored in a `pfiles` folder the user's home
directory. When tasks are run in parallel, care is needed to ensure parallel tasks
don't use the same parameter files (and hence be called with the same parameters).

HEASoftPy provides and a content utility that allows tasks to run using temporary
parameter files, so parallel runs remain independent.

The following is an example, we show how to run a `nicerl2` to process NICER event
files from many observations in parallel (the data themselves are downloaded in
the 'configuration' cell near the top of the notebook).

We do this by creating a helper function `worker` that wraps the task call and add
the temporary parameter files (see the 'Global Setup: Functions' section near the
top of this notebook).

The `NUM_CORES` constant (defined in the 'Global Setup: Configuration' subsection
near the top of this notebook) controls the number of processes to run in parallel.

```{danger}
Running the `nicerl2` tool requires that up-to-date geomagnetic data are available on
your system; [see this for a discussion](https://heasarc.gsfc.nasa.gov/docs/nicer/analysis_threads/geomag/).
The path to the geomagnetic data can either be set in the GEOMAG_PATH environment
variable, or passed to the tool directly through the `geomag_path` parameter.
 ```

We download the geomagnetic data using a HEASoft tool; `nigeodown`. Once again we
use the Python interface provided by HEASoftPy.

In this case, we have wrapped the
`nicerl2` HEASoftPy call in another function, to make parallelization easier. Rather
than adding another argument to the `worker` function (defined near the top of this
notebook), and passing the geomagnetic data path, or setting
an environment variable, we define a constant global variable that we read in the `worker` function,

In [18]:
GEOMAG_PATH = os.path.join(ROOT_DATA_DIR, "geomag")
out = hsp.nigeodown(outdir=GEOMAG_PATH, allow_failure=False, clobber=True)

In [19]:
os.listdir(GEOMAG_PATH)

['.tmpdir',
 'dst_kyoto.fits',
 'f107_petincton.fits',
 'kp_noaa.fits',
 'kp_potsdam.fits',
 'solarphi_oulu.fits']

This geomagnetic data is going to help us process the following NICER observations:

In [20]:
NI_OBS_IDS

['1010010121', '1010010122', '1012020112', '1012020114']

Now, we can run the parallelized `nicerl2` tasks:

In [21]:
with mp.Pool(NUM_CORES) as p:
    obsids = [os.path.join(ni_data_dir, oi) for oi in NI_OBS_IDS]
    result = p.map(worker, obsids)

# Show the output of the parallel tasks
result

[---------------------
 :: Execution Result ::
 ---------------------
 > Return Code: 0
 > Output:
 nicerl2 1.41
 --------------------------------------------------------
          Requested task operations: MKF,SCREEN,CALMERGE
        Processed prefilter_columns: NICERV6
    Processed niprefilter2_coltypes: NICERV6,BASE
           Processed geomag_columns: kp_potsdam.fits(KP),COR_NYM,solarphi_oulu.fits(SOLAR_PHI)
   Inputs:
   (INDIR) /home/jovyan/project/_data/NICER/1010010121
   (UF)    /home/jovyan/project/_data/NICER/1010010121/xti/event_uf
   (CL)    /home/jovyan/project/_data/NICER/1010010121/xti/event_cl
   (MKF)   /home/jovyan/project/_data/NICER/1010010121/auxil/ni1010010121.mkf
 
                         MPU List: 0-6
                           OBS_ID: 1010010121
                           OBJECT: 1E_0102.2-7219
                         Position: RA    16.0080  Dec   -72.0310 [deg] OBJECT
                                   RA    16.0080  Dec   -72.0300 [deg] NOMINAL
        

In this particular case, we've run `nicerl2` in such a way that the outputs are placed in the
original downloaded data directories, overwriting any existing files with newer versions.

We can quickly examine the cleaned events directory of one of the NICER observations:

In [22]:
os.listdir(os.path.join(ni_data_dir, "1012020112", "xti", "event_cl"))

['ni1012020112_0mpu0_ufa.evt',
 'ni1012020112_0mpu1_ufa.evt',
 'ni1012020112_0mpu2_ufa.evt',
 'ni1012020112_0mpu3_ufa.evt',
 'ni1012020112_0mpu4_ufa.evt',
 'ni1012020112_0mpu5_ufa.evt',
 'ni1012020112_0mpu6_ufa.evt',
 'ni1012020112_0mpu7_ufa.evt',
 'ni1012020112_0mpu7_cl.evt']

## About this Notebook

Author: Abdu Zoghbi, HEASARC Staff Scientist

Author: David Turner, HEASARC Staff Scientist

Updated On: 2026-02-03

### Additional Resources

For more documentation on using HEASoft see :

- [HEASoftPy HEASARC page](https://heasarc.gsfc.nasa.gov/docs/software/lheasoft/heasoftpy/)
- [HEASoft HEASARC page](https://heasarc.gsfc.nasa.gov/docs/software/lheasoft/)
- [HEASoftPy GitHub](https://github.com/HEASARC/heasoftpy)

### Acknowledgements

### References